In [19]:
import numpy as np 
from sklearn.linear_model import LinearRegression, Ridge, Lasso

In [14]:
def tune_bb(algo, X, y, 
            regularization="Dropout", M=10, 
            c=None, K=5, criterion="MSE"):

    """function to automatically tune blackbox regression model
    
    Parameters:
    -----------

    algo : callable
        A learning algorithm that takes as input a matrix X in R nxp
        and a vector of responses Y in Rn and returns a function that
        maps inputs to outputs.
    X : array-like of shape (n,p)
        training data X in R nxp
    y : array-like of shape (n,)
        training labels, Y, in Rn
    regularization : str, default="Dropout"
        regularization method, can be any of "Dropout",
        "NoiseAddition", or "Robust"
    M : int, default=
        A positive integer indicating the number of Monte Carlo
        replicates to be used if the method specified is Dropout or 
        NoiseAddition
    c : default=None
        A vector of column bounds to be used if method specified is "Robust"
    K : int, default=5
        A positive integer indicating the number of CV-folds to be used to 
        tune the amount of regularization, e.g., K = 5 indicates five-fold CV
    criterion : str, default="MSE"
        A criterion to be used to evaluate the method that belongs to the set 
        {MSE, MAD} where MSE encodes mean square error and MAD encodes mean
        absolute deviation.

    Returns:
    -----------
    tuned_mode : callable 
        A tuned predictive model that optimizes the specific criterion using 
        the specified method

    Example:
    -----------
    >>> tune_bb()
    >>> 
    """
    prod = np.dot(X,y)
    return prod
    
    
    

In [4]:
# matrices to play around with and feed into function 

X = np.array([[1,5,4],[3,4,5],[6,4,3]])
y = np.array([[1,2,5]]).T

print(y.shape) # 3x1
print(X.shape) # 3x3

(3, 1)
(3, 3)


In [28]:
tune_bb(4,X,y)

array([[31],
       [36],
       [29]])

In [15]:
# maybe something like this if they all follow the .fit() syntax? 
def model_test(model,X,y):
    return model.fit(X,y)

In [ ]:
# this passes the model in ?
model_test(LinearRegression(), X, y).coef_

In [18]:
model_test(Ridge(), X, y).coef_ # niceeeeeeeeee

array([[ 0.67567568, -0.05405405, -0.32432432]])

In [20]:
model_test(Lasso(), X, y).coef_

array([ 0.57894737, -0.        , -0.        ])

In [21]:
type(Lasso()) # check that all models are from sklearn or straight to jail

sklearn.linear_model._coordinate_descent.Lasso